# 🚀 XAU-SNIPER Training (GitHub + Drive Workflow)

Este notebook integra tu código en GitHub con tus datos/almacenamiento en Google Drive.

### Requisitos:
1. **GitHub:** El código (train.py, env, etc.) se descarga automáticamente.
2. **Google Drive:** 
    - Crea una carpeta llamada `GLDIA_DATA` en la raíz de tu Drive.
    - Sube el archivo gigante `xauusd_m1.csv` dentro de `GLDIA_DATA`.
    - Sube `xauusd_m5.csv` y `xauusd_m15.csv` también si quieres.
    - (Opcional) Los resultados se guardarán automáticamente en `GLDIA_DATA/outputs`.


In [ ]:
# 1. Instalar Librerías
!pip install stable-baselines3[extra] sb3-contrib pandas numpy ta shimmy gymnasium

In [ ]:
# 2. Clonar Repositorio (Código Reciente)
import os
if os.path.exists('gldia'):
    %cd gldia
    !git pull
else:
    !git clone https://github.com/alejoamorocho/gldia.git
    %cd gldia

In [ ]:
# 3. Conectar Google Drive (para Datos y Outputs)
from google.colab import drive
drive.mount('/content/drive')

# Rutas
DRIVE_ROOT = '/content/drive/MyDrive/GLDIA_DATA'
REPO_DATA = '/content/gldia/data'
REPO_OUTPUTS = '/content/gldia/outputs'

# Crear carpeta en Drive si no existe
if not os.path.exists(DRIVE_ROOT):
    os.makedirs(DRIVE_ROOT)
    print(f"Carpeta creada en Drive: {DRIVE_ROOT}. ¡Sube aquí tus CSVs gigantes!")

# Enlazar Datos (Symlink)
# El código busca datos en 'data/'. Haremos que 'data/' tome los archivos de Drive.
# Específicamente los archivos M1 que no están en GitHub.
import shutil

files_to_link = ['xauusd_m1.csv', 'xauusd_m5.csv', 'xauusd_m15.csv']

for f in files_to_link:
    src = os.path.join(DRIVE_ROOT, f)
    dst = os.path.join(REPO_DATA, f)
    if os.path.exists(src):
        if os.path.exists(dst):
            os.remove(dst)
        os.symlink(src, dst)
        print(f"Enlazado: {f} -> Drive")
    else:
        print(f"Aviso: {f} no encontrado en Drive ({DRIVE_ROOT}). Si lo necesitas, súbelo.")

# Enlazar Outputs (Para que los modelos se guarden en Drive y no se pierdan)
# Movemos la carpeta outputs del repo a Drive (o creamos symlink)

if not os.path.exists(os.path.join(DRIVE_ROOT, 'outputs')):
    os.makedirs(os.path.join(DRIVE_ROOT, 'outputs'))

# Redirigir outputs del script a Drive
# Simplemente pasaremos --output_dir al script de entrenamiento
drive_output_path = os.path.join(DRIVE_ROOT, 'outputs')

In [ ]:
# 4. 🐉 EJECUTAR ENTRENAMIENTO
# Usamos la ruta de Drive para guardar todo.

!python training/train.py --timesteps 5000000 --output_dir "{drive_output_path}"